# Imports

In [ ]:
!pip install yfinance --upgrade --no-cache-dir
!pip install finnhub-python
!pip3 install news-please

!pip install torch
!pip install peft
!pip install -U accelerate
!pip install transformers bitsandbytes
!pip install pynvml
!pip install gnews
!pip install selenium
!pip install webdriver_manager
!pip install --upgrade undetected-chromedriver

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
stock_news = [{'title': "Tesla Stock Has Doubled Since Barron’s Said to Buy It Last Year. What to Do Now. - Barron's", 'description': "Tesla Stock Has Doubled Since Barron’s Said to Buy It Last Year. What to Do Now.  Barron's", 'published date': 'Fri, 05 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiRGh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLWJ1eS1zZWxsLTUxOTI0ZDRm0gFIaHR0cHM6Ly93d3cuYmFycm9ucy5jb20vYW1wL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLWJ1eS1zZWxsLTUxOTI0ZDRm?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Short sellers lost more money betting against Tesla than any other company last year - CNN', 'description': 'Short sellers lost more money betting against Tesla than any other company last year  CNN', 'published date': 'Sat, 06 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiTWh0dHBzOi8vd3d3LmNubi5jb20vMjAyNC8wMS8wNi9idXNpbmVzcy90ZXNsYS1zaG9ydC1zZWxsZXJzLWxvc3Nlcy9pbmRleC5odG1s0gFRaHR0cHM6Ly9hbXAuY25uLmNvbS9jbm4vMjAyNC8wMS8wNi9idXNpbmVzcy90ZXNsYS1zaG9ydC1zZWxsZXJzLWxvc3Nlcy9pbmRleC5odG1s?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.cnn.com', 'title': 'CNN'}}, {'title': "Tesla Stock Is on a Bad Streak. It’s Probably Taxes More Than Elon Musk. - Barron's", 'description': "Tesla Stock Is on a Bad Streak. It’s Probably Taxes More Than Elon Musk.  Barron's", 'published date': 'Tue, 09 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiR2h0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXRheGVzLWxvd2VyLXByaWNlLTA3YjkxMTQy0gFLaHR0cHM6Ly93d3cuYmFycm9ucy5jb20vYW1wL2FydGljbGVzL3Rlc2xhLXN0b2NrLXRheGVzLWxvd2VyLXByaWNlLTA3YjkxMTQy?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': "Tesla Stock Is Being Sold by Almost Everyone. Cathie Wood Is Buying. - Barron's", 'description': "Tesla Stock Is Being Sold by Almost Everyone. Cathie Wood Is Buying.  Barron's", 'published date': 'Sat, 13 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLWNhdGhpZS13b29kLWJ1eS1zZWxsLWI0ZDg4NTdj0gFOaHR0cHM6Ly93d3cuYmFycm9ucy5jb20vYW1wL2FydGljbGVzL3Rlc2xhLXN0b2NrLWNhdGhpZS13b29kLWJ1eS1zZWxsLWI0ZDg4NTdj?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': "Tesla Stock Edges Higher After Elon Musk's Comments on Ownership, AI - The Wall Street Journal", 'description': "Tesla Stock Edges Higher After Elon Musk's Comments on Ownership, AI  The Wall Street Journal", 'published date': 'Tue, 16 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMisQFodHRwczovL3d3dy53c2ouY29tL2xpdmVjb3ZlcmFnZS9zdG9jay1tYXJrZXQtdG9kYXktZG93LWpvbmVzLWJhbmstZWFybmluZ3MtMDEtMTYtMjAyNC9jYXJkL3Rlc2xhLXN0b2NrLXNsaXBzLWFmdGVyLWVsb24tbXVzay1zLWNvbW1lbnRzLW9uLWFpLWFuZC1vd25lcnNoaXAtclBuWXhmTjB4ZmNEOTJEV1RuVU3SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.wsj.com', 'title': 'The Wall Street Journal'}}, {'title': "Tesla Stock Falls on More Price Cuts - Barron's", 'description': "Tesla Stock Falls on More Price Cuts  Barron's", 'published date': 'Wed, 17 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiR2h0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLW1vZGVsLXktcHJpY2UtY3V0LWJlZjA2NTY00gFLaHR0cHM6Ly93d3cuYmFycm9ucy5jb20vYW1wL2FydGljbGVzL3Rlc2xhLXN0b2NrLW1vZGVsLXktcHJpY2UtY3V0LWJlZjA2NTY0?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Tesla Stock Falls After Slashing Prices in Europe - Investopedia', 'description': 'Tesla Stock Falls After Slashing Prices in Europe  Investopedia', 'published date': 'Wed, 17 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiVmh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vdGVzbGEtc3RvY2stZmFsbHMtYWZ0ZXItc2xhc2hpbmctcHJpY2VzLWluLWV1cm9wZS04NDMwNjU30gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investopedia.com', 'title': 'Investopedia'}}, {'title': "EV Bloodbath Continues. Tesla Stock Set for Disappointing Record. - Barron's", 'description': "EV Bloodbath Continues. Tesla Stock Set for Disappointing Record.  Barron's", 'published date': 'Fri, 19 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL2V2LXN0b2Nrcy10ZXNsYS1kaXNhcHBvaW50aW5nLXJlY29yZC02NDQ0N2Y3ONIBUmh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FtcC9hcnRpY2xlcy9ldi1zdG9ja3MtdGVzbGEtZGlzYXBwb2ludGluZy1yZWNvcmQtNjQ0NDdmNzg?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': "Time to Buy the Recent Dip in Tesla's Stock Before Earnings? - Yahoo Finance", 'description': "Time to Buy the Recent Dip in Tesla's Stock Before Earnings?  Yahoo Finance", 'published date': 'Mon, 22 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy90aW1lLWJ1eS1yZWNlbnQtZGlwLXRlc2xhcy0yMTAyMDAyOTIuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'Top analyst unveils new Tesla stock price target ahead of earnings - TheStreet', 'description': 'Top analyst unveils new Tesla stock price target ahead of earnings  TheStreet', 'published date': 'Mon, 22 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMibGh0dHBzOi8vd3d3LnRoZXN0cmVldC5jb20vaW52ZXN0aW5nL3N0b2Nrcy90b3AtYW5hbHlzdC1jdXRzLXRlc2xhLXByaWNlLXRhcmdldC1haGVhZC1vZi1jYXV0aW91cy1xNC1lYXJuaW5nc9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.thestreet.com', 'title': 'TheStreet'}}, {'title': 'Tesla Stock Slips After Q4 Revenue, Earnings Miss Estimates - The Wall Street Journal', 'description': 'Tesla Stock Slips After Q4 Revenue, Earnings Miss Estimates  The Wall Street Journal', 'published date': 'Wed, 24 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMijwFodHRwczovL3d3dy53c2ouY29tL2xpdmVjb3ZlcmFnZS9zdG9jay1tYXJrZXQtdG9kYXktZG93LWpvbmVzLWVhcm5pbmdzLTAxLTI0LTIwMjQvY2FyZC90ZXNsYS1zdG9jay1nYWlucy1haGVhZC1vZi1lYXJuaW5ncy0xVkE4MGxCTVo5b2kzdG1rNTFnbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.wsj.com', 'title': 'The Wall Street Journal'}}, {'title': 'Tesla Stock Falls As Earnings Come In Short Of Market Expectations - Investopedia', 'description': 'Tesla Stock Falls As Earnings Come In Short Of Market Expectations  Investopedia', 'published date': 'Wed, 24 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiYGh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vdGVzbGEtc3RvY2stZmFsbHMtYXMtZWFybmluZ3MtY29tZS1pbi1zaG9ydC1vZi1leHBlY3RhdGlvbnMtODU0OTI2ONIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investopedia.com', 'title': 'Investopedia'}}, {'title': 'Massive News for Tesla Stock Investors! - Yahoo Finance', 'description': 'Massive News for Tesla Stock Investors!  Yahoo Finance', 'published date': 'Wed, 24 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiUGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9tYXNzaXZlLW5ld3MtdGVzbGEtc3RvY2staW52ZXN0b3JzLTIzNTMyNzM5Ny5odG1s0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'Tesla stock plummets more than 10% after earnings miss expectations - ABC News', 'description': 'Tesla stock plummets more than 10% after earnings miss expectations  ABC News', 'published date': 'Thu, 25 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiaGh0dHBzOi8vYWJjbmV3cy5nby5jb20vQnVzaW5lc3MvdGVzbGEtc3RvY2stcGx1bW1ldHMtYWZ0ZXItZWFybmluZ3MtbWlzcy1leHBlY3RhdGlvbnMvc3Rvcnk_aWQ9MTA2Njc3NDA10gFsaHR0cHM6Ly9hYmNuZXdzLmdvLmNvbS9hbXAvQnVzaW5lc3MvdGVzbGEtc3RvY2stcGx1bW1ldHMtYWZ0ZXItZWFybmluZ3MtbWlzcy1leHBlY3RhdGlvbnMvc3Rvcnk_aWQ9MTA2Njc3NDA1?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://abcnews.go.com', 'title': 'ABC News'}}, {'title': "Tesla Stock Will Head Lower From Here. The Charts Don’t Lie. - Barron's", 'description': "Tesla Stock Will Head Lower From Here. The Charts Don’t Lie.  Barron's", 'published date': 'Thu, 25 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLWVhcm5pbmdzLW11c2stMWYyMjlhNmbSAU1odHRwczovL3d3dy5iYXJyb25zLmNvbS9hbXAvYXJ0aWNsZXMvdGVzbGEtc3RvY2stcHJpY2UtZWFybmluZ3MtbXVzay0xZjIyOWE2Zg?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Tesla Shares Tumble As Growth Stalls - The New York Times', 'description': 'Tesla Shares Tumble As Growth Stalls  The New York Times', 'published date': 'Thu, 25 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiVWh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDEvMjUvYnVzaW5lc3MvZGVhbGJvb2svdGVzbGEtc2hhcmVzLW91dGxvb2stZ3Jvd3RoLmh0bWzSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.nytimes.com', 'title': 'The New York Times'}}, {'title': 'Tesla shares close down 12% after automaker warns of slowdown - CNBC', 'description': 'Tesla shares close down 12% after automaker warns of slowdown  CNBC', 'published date': 'Thu, 25 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiZ2h0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDEvMjUvdGVzbGEtdHNsYS1zaGFyZXMtZmFsbC1hZnRlci1tdXNrcy1ldi1tYWtlci13YXJucy1vZi0yMDI0LXNsb3dkb3duLmh0bWzSAWtodHRwczovL3d3dy5jbmJjLmNvbS9hbXAvMjAyNC8wMS8yNS90ZXNsYS10c2xhLXNoYXJlcy1mYWxsLWFmdGVyLW11c2tzLWV2LW1ha2VyLXdhcm5zLW9mLTIwMjQtc2xvd2Rvd24uaHRtbA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.cnbc.com', 'title': 'CNBC'}}, {'title': 'Tesla Stock Heads Toward 8-Month Low After 23% Annual Profit Decline - Forbes', 'description': 'Tesla Stock Heads Toward 8-Month Low After 23% Annual Profit Decline  Forbes', 'published date': 'Thu, 25 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMidmh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvZGVyZWtzYXVsLzIwMjQvMDEvMjUvdGVzbGEtc3RvY2staGVhZHMtdG93YXJkLTgtbW9udGgtbG93LWFmdGVyLTIzLWFubnVhbC1wcm9maXQtZGVjbGluZS_SAXpodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2RlcmVrc2F1bC8yMDI0LzAxLzI1L3Rlc2xhLXN0b2NrLWhlYWRzLXRvd2FyZC04LW1vbnRoLWxvdy1hZnRlci0yMy1hbm51YWwtcHJvZml0LWRlY2xpbmUvYW1wLw?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.forbes.com', 'title': 'Forbes'}}, {'title': 'Tesla Stock Plunges as 2024 Volume Growth Hits the Brakes—Key Price Levels to Monitor - Investopedia', 'description': 'Tesla Stock Plunges as 2024 Volume Growth Hits the Brakes—Key Price Levels to Monitor  Investopedia', 'published date': 'Thu, 25 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiemh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vdGVzbGEtc3RvY2stcGx1bmdlcy1hcy0yMDI0LXZvbHVtZS1ncm93dGgtaGl0cy10aGUtYnJha2VzLWtleS1wcmljZS1sZXZlbHMtdG8tbW9uaXRvci04NTQ5NTU40gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investopedia.com', 'title': 'Investopedia'}}, {'title': "Tesla stock falls 12% as EV maker warns production growth will be 'notably lower' than 2023 - Yahoo Finance", 'description': "Tesla stock falls 12% as EV maker warns production growth will be 'notably lower' than 2023  Yahoo Finance", 'published date': 'Thu, 25 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMihgFodHRwczovL2ZpbmFuY2UueWFob28uY29tL25ld3MvdGVzbGEtc3RvY2stZmFsbHMtMTItYXMtZXYtbWFrZXItd2FybnMtcHJvZHVjdGlvbi1ncm93dGgtd2lsbC1iZS1ub3RhYmx5LWxvd2VyLXRoYW4tMjAyMy0yMTAxNTQ4NTIuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'Why Tesla Stock Plummeted Today - The Motley Fool', 'description': 'Why Tesla Stock Plummeted Today  The Motley Fool', 'published date': 'Thu, 25 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAxLzI1L3doeS10ZXNsYS1zdG9jay1wbHVtbWV0ZWQtdG9kYXkv0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': 'Tesla share plunge wipes out $80 billion in market value, after dour earnings call - CNN', 'description': 'Tesla share plunge wipes out $80 billion in market value, after dour earnings call  CNN', 'published date': 'Fri, 26 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiUGh0dHBzOi8vd3d3LmNubi5jb20vMjAyNC8wMS8yNS9pbnZlc3RpbmcvdGVzbGEtc3RvY2stZHJvcC1jaGluYS1zYWxlcy9pbmRleC5odG1s0gFUaHR0cHM6Ly9hbXAuY25uLmNvbS9jbm4vMjAyNC8wMS8yNS9pbnZlc3RpbmcvdGVzbGEtc3RvY2stZHJvcC1jaGluYS1zYWxlcy9pbmRleC5odG1s?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.cnn.com', 'title': 'CNN'}}, {'title': "Tesla Stock Will Have to Wait for a Rebound, Bull Says. Blame the Rule of Three. - Barron's", 'description': "Tesla Stock Will Have to Wait for a Rebound, Bull Says. Blame the Rule of Three.  Barron's", 'published date': 'Fri, 26 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSGh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLWVhcm5pbmdzLWV2cy1zYWxlcy1lN2UwYjk5YtIBTGh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FtcC9hcnRpY2xlcy90ZXNsYS1zdG9jay1lYXJuaW5ncy1ldnMtc2FsZXMtZTdlMGI5OWI?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Tesla Stock is the Biggest Laggard in the Magnificent Seven - The Wall Street Journal', 'description': 'Tesla Stock is the Biggest Laggard in the Magnificent Seven  The Wall Street Journal', 'published date': 'Fri, 26 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMipwFodHRwczovL3d3dy53c2ouY29tL2xpdmVjb3ZlcmFnZS9zdG9jay1tYXJrZXQtdG9kYXktZG93LWpvbmVzLWVhcm5pbmdzLTAxLTI2LTIwMjQvY2FyZC90ZXNsYS1zdG9jay1pcy10aGUtYmlnZ2VzdC1sYWdnYXJkLWluLXRoZS1tYWduaWZpY2VudC1zZXZlbi1xam95VGVramlicUtCTHdPV1M1SdIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.wsj.com', 'title': 'The Wall Street Journal'}}, {'title': "Where Will Tesla's Stock Price Be in 2 Years? - Yahoo Finance", 'description': "Where Will Tesla's Stock Price Be in 2 Years?  Yahoo Finance", 'published date': 'Sat, 27 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy93aGVyZS10ZXNsYXMtc3RvY2stcHJpY2UtMi0yMDUxNDI1NjYuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'Is Tesla still a buy? One CIO weighs in and names 3 alternatives he likes - CNBC', 'description': 'Is Tesla still a buy? One CIO weighs in and names 3 alternatives he likes  CNBC', 'published date': 'Sun, 28 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiaGh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDEvMjkvaXMtdGVzbGEtc3RpbGwtYS1idXktd2hhdC10aGlzLWNpby1zYXlzLWFuZC0zLWFsdGVybmF0aXZlcy1oZS1saWtlcy5odG1s0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.cnbc.com', 'title': 'CNBC'}}, {'title': 'Will Tesla Stock Reach $100? - Yahoo Finance', 'description': 'Will Tesla Stock Reach $100?  Yahoo Finance', 'published date': 'Mon, 29 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQ2h0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy90ZXNsYS1zdG9jay1yZWFjaC0xMDAtMTYwMDAwOTk5Lmh0bWzSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'Tesla Stock Rises as EV Maker Unveils Big Spending Plans on Various Projects, Including AI - Investopedia', 'description': 'Tesla Stock Rises as EV Maker Unveils Big Spending Plans on Various Projects, Including AI  Investopedia', 'published date': 'Mon, 29 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMifmh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vdGVzbGEtc3RvY2stcmlzZXMtYXMtZXYtbWFrZXItdW52ZWlscy1iaWctc3BlbmRpbmctcGxhbnMtb24tdmFyaW91cy1wcm9qZWN0cy1pbmNsdWRpbmctYWktODU1MTYxMNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investopedia.com', 'title': 'Investopedia'}}, {'title': "What's the Forecast for Tesla Stock in 2024? - Nasdaq", 'description': "What's the Forecast for Tesla Stock in 2024?  Nasdaq", 'published date': 'Tue, 30 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vd3d3Lm5hc2RhcS5jb20vYXJ0aWNsZXMvd2hhdHMtdGhlLWZvcmVjYXN0LWZvci10ZXNsYS1zdG9jay1pbi0yMDI00gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.nasdaq.com', 'title': 'Nasdaq'}}, {'title': '2 Elon Musk Predictions Tesla Stock Investors Need To Hear - Yahoo Finance', 'description': '2 Elon Musk Predictions Tesla Stock Investors Need To Hear  Yahoo Finance', 'published date': 'Tue, 30 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiS2h0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy8yLWVsb24tbXVzay1wcmVkaWN0aW9ucy10ZXNsYS0xMDM1MDA1NDAuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': "Tesla Stock Is Falling. Sell It Now, Says Analyst. - Barron's", 'description': "Tesla Stock Is Falling. Sell It Now, Says Analyst.  Barron's", 'published date': 'Wed, 31 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQGh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLXNlbGwtMWVmOTQ3ZGPSAURodHRwczovL3d3dy5iYXJyb25zLmNvbS9hbXAvYXJ0aWNsZXMvdGVzbGEtc3RvY2stcHJpY2Utc2VsbC0xZWY5NDdkYw?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': "Judge voids Elon Musk's 'unfathomable' $56 billion Tesla pay package - Reuters", 'description': "Judge voids Elon Musk's 'unfathomable' $56 billion Tesla pay package  Reuters", 'published date': 'Wed, 31 Jan 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiamh0dHBzOi8vd3d3LnJldXRlcnMuY29tL2xlZ2FsL2p1ZGdlLXJ1bGVzLWZhdm9yLXBsYWludGlmZnMtY2hhbGxlbmdpbmctbXVza3MtdGVzbGEtcGF5LXBhY2thZ2UtMjAyNC0wMS0zMC_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.reuters.com', 'title': 'Reuters'}}, {'title': "What Tesla Stock Volatility Says About Tech's 'Magnificent Seven' - The Wall Street Journal", 'description': "What Tesla Stock Volatility Says About Tech's 'Magnificent Seven'  The Wall Street Journal", 'published date': 'Fri, 02 Feb 2024 05:02:46 GMT', 'url': 'https://news.google.com/rss/articles/CBMilAFodHRwczovL3d3dy53c2ouY29tL3ZpZGVvL3Nlcmllcy9uZXdzLWV4cGxhaW5lcnMvd2hhdC10ZXNsYS1zdG9jay12b2xhdGlsaXR5LXNheXMtYWJvdXQtdGVjaHMtbWFnbmlmaWNlbnQtc2V2ZW4vNURCRDgyOEEtRDhBQy00M0JELUIyQ0MtN0FCMTY1RUQxRDgz0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.wsj.com', 'title': 'The Wall Street Journal'}}, {'title': "Video: What Tesla Stock Volatility Says About the 'Magnificent Seven' - The Wall Street Journal", 'description': "Video: What Tesla Stock Volatility Says About the 'Magnificent Seven'  The Wall Street Journal", 'published date': 'Fri, 02 Feb 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMirwFodHRwczovL3d3dy53c2ouY29tL2xpdmVjb3ZlcmFnZS9zdG9jay1tYXJrZXQtdG9kYXktZG93LWpvbmVzLWVhcm5pbmdzLTAyLTAyLTIwMjQvY2FyZC92aWRlby13aGF0LXRlc2xhLXN0b2NrLXZvbGF0aWxpdHktc2F5cy1hYm91dC10aGUtbWFnbmlmaWNlbnQtc2V2ZW4tLTVrRjB2M0tZNUdtQWRUTll5Zkxk0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.wsj.com', 'title': 'The Wall Street Journal'}}, {'title': "Tesla Stock Is Down. Recalls Over Font Size Aren’t Why. - Barron's", 'description': "Tesla Stock Is Down. Recalls Over Font Size Aren’t Why.  Barron's", 'published date': 'Fri, 02 Feb 2024 08:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQmh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLXJlY2FsbC1hZjMwMzRlN9IBRmh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FtcC9hcnRpY2xlcy90ZXNsYS1zdG9jay1wcmljZS1yZWNhbGwtYWYzMDM0ZTc?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Massive News for Tesla Stock Investors - Yahoo Finance', 'description': 'Massive News for Tesla Stock Investors  Yahoo Finance', 'published date': 'Sun, 04 Feb 2024 15:30:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiUGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9tYXNzaXZlLW5ld3MtdGVzbGEtc3RvY2staW52ZXN0b3JzLTE1MzAwMDM2My5odG1s0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'Massive News for Tesla Stock Investors - The Motley Fool', 'description': 'Massive News for Tesla Stock Investors  The Motley Fool', 'published date': 'Sun, 04 Feb 2024 15:30:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiUWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzA0L21hc3NpdmUtbmV3cy1mb3ItdGVzbGEtc3RvY2staW52ZXN0b3JzL9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': 'Analyst who correctly warned Tesla stock could fall unveils new target - TheStreet', 'description': 'Analyst who correctly warned Tesla stock could fall unveils new target  TheStreet', 'published date': 'Mon, 05 Feb 2024 02:00:40 GMT', 'url': 'https://news.google.com/rss/articles/CBMia2h0dHBzOi8vd3d3LnRoZXN0cmVldC5jb20vYXV0b21vdGl2ZS9hbmFseXN0LXdoby1jb3JyZWN0bHktd2FybmVkLXRlc2xhLXN0b2NrLWNvdWxkLWZhbGwtdW52ZWlscy1uZXctdGFyZ2V00gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.thestreet.com', 'title': 'TheStreet'}}, {'title': 'Will AMD Be Worth More Than Tesla by 2025? - The Motley Fool', 'description': 'Will AMD Be Worth More Than Tesla by 2025?  The Motley Fool', 'published date': 'Mon, 05 Feb 2024 10:53:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiVGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzA1L3dpbGwtYW1kLWJlLXdvcnRoLW1vcmUtdGhhbi10ZXNsYS1ieS0yMDI1L9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': 'Tesla’s stock dive means the EV maker is now worth less than Broadcom - MarketWatch', 'description': 'Tesla’s stock dive means the EV maker is now worth less than Broadcom  MarketWatch', 'published date': 'Mon, 05 Feb 2024 15:02:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiYGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9zYXAtdG8tc3RvcC11c2luZy10ZXNsYS1mb3ItY29tcGFueS1mbGVldC1yZXBvcnQtc2F5cy01NDYyN2RiYtIBZGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9hbXAvc3Rvcnkvc2FwLXRvLXN0b3AtdXNpbmctdGVzbGEtZm9yLWNvbXBhbnktZmxlZXQtcmVwb3J0LXNheXMtNTQ2MjdkYmI?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.marketwatch.com', 'title': 'MarketWatch'}}, {'title': "Tesla Stock Is Falling. Here’s What’s Dragging It Down. - Barron's", 'description': "Tesla Stock Is Falling. Here’s What’s Dragging It Down.  Barron's", 'published date': 'Mon, 05 Feb 2024 16:52:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLXRvZGF5LTBjNDEyMWRi0gFFaHR0cHM6Ly93d3cuYmFycm9ucy5jb20vYW1wL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLXRvZGF5LTBjNDEyMWRi?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': "ON Semi Stock Rises. Guidance Was Good Enough but Not Great for Tesla. - Barron's", 'description': "ON Semi Stock Rises. Guidance Was Good Enough but Not Great for Tesla.  Barron's", 'published date': 'Mon, 05 Feb 2024 17:09:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiTWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL29uLXNlbWktc3RvY2stZWFybmluZ3MtZXZzLWd1aWRhbmNlLTM5ZjIyNjhm0gFRaHR0cHM6Ly93d3cuYmFycm9ucy5jb20vYW1wL2FydGljbGVzL29uLXNlbWktc3RvY2stZWFybmluZ3MtZXZzLWd1aWRhbmNlLTM5ZjIyNjhm?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Tesla falls on SAP snub report, Piper Sandler price target cut - Yahoo Finance', 'description': 'Tesla falls on SAP snub report, Piper Sandler price target cut  Yahoo Finance', 'published date': 'Mon, 05 Feb 2024 17:19:06 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSWh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy90ZXNsYS1mYWxscy1yZXBvcnQtc2FwLXNudWItMTcxOTA2NDU3Lmh0bWzSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'Elon Musk net worth plummets as Tesla stock, market cap fall - Quartz', 'description': 'Elon Musk net worth plummets as Tesla stock, market cap fall  Quartz', 'published date': 'Mon, 05 Feb 2024 17:20:25 GMT', 'url': 'https://news.google.com/rss/articles/CBMiOWh0dHBzOi8vcXouY29tL2Vsb24tbXVzay1uZXQtd29ydGgtdGVzbGEtc3RvY2stMTg1MTIyNTMwOdIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://qz.com', 'title': 'Quartz'}}, {'title': 'Why Tesla Stock Tanked Monday - The Motley Fool', 'description': 'Why Tesla Stock Tanked Monday  The Motley Fool', 'published date': 'Mon, 05 Feb 2024 19:04:28 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzA1L3doeS10ZXNsYS1zdG9jay10YW5rZWQtbW9uZGF5L9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': "Tesla Stock Approaches 8-Month Low Amid Scrutiny of Directors' Ties to Musk - Investopedia", 'description': "Tesla Stock Approaches 8-Month Low Amid Scrutiny of Directors' Ties to Musk  Investopedia", 'published date': 'Mon, 05 Feb 2024 19:35:32 GMT', 'url': 'https://news.google.com/rss/articles/CBMib2h0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vdGVzbGEtc3RvY2stYXBwcm9hY2hlcy04LW1vbnRoLWxvdy1hbWlkLXNjcnV0aW55LW9mLWRpcmVjdG9ycy10aWVzLXRvLW11c2stODU1ODg5ONIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investopedia.com', 'title': 'Investopedia'}}, {'title': 'Tesla Stock Sinks To 9-Month Low—Worst S&P 500 Performer—As Musk Controversies Leave ‘Black Cloud’ - Forbes', 'description': 'Tesla Stock Sinks To 9-Month Low—Worst S&P 500 Performer—As Musk Controversies Leave ‘Black Cloud’  Forbes', 'published date': 'Mon, 05 Feb 2024 20:01:39 GMT', 'url': 'https://news.google.com/rss/articles/CBMikgFodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2RlcmVrc2F1bC8yMDI0LzAyLzA1L3Rlc2xhLXN0b2NrLXNpbmtzLXRvLTktbW9udGgtbG93LXdvcnN0LXNwLTUwMC1wZXJmb3JtZXItYXMtbXVzay1jb250cm92ZXJzaWVzLWxlYXZlLWJsYWNrLWNsb3VkL9IBlgFodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2RlcmVrc2F1bC8yMDI0LzAyLzA1L3Rlc2xhLXN0b2NrLXNpbmtzLXRvLTktbW9udGgtbG93LXdvcnN0LXNwLTUwMC1wZXJmb3JtZXItYXMtbXVzay1jb250cm92ZXJzaWVzLWxlYXZlLWJsYWNrLWNsb3VkL2FtcC8?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.forbes.com', 'title': 'Forbes'}}, {'title': 'Tesla stock drops as investors scrutinize board’s close personal ties to Elon Musk - CNN', 'description': 'Tesla stock drops as investors scrutinize board’s close personal ties to Elon Musk  CNN', 'published date': 'Mon, 05 Feb 2024 20:53:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vd3d3LmNubi5jb20vMjAyNC8wMi8wNS9idXNpbmVzcy90ZXNsYS1zdG9jay1lbG9uLW11c2stYm9hcmQvaW5kZXguaHRtbNIBUmh0dHBzOi8vYW1wLmNubi5jb20vY25uLzIwMjQvMDIvMDUvYnVzaW5lc3MvdGVzbGEtc3RvY2stZWxvbi1tdXNrLWJvYXJkL2luZGV4Lmh0bWw?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.cnn.com', 'title': 'CNN'}}, {'title': 'Why Tesla Stock Tanked in January - The Motley Fool', 'description': 'Why Tesla Stock Tanked in January  The Motley Fool', 'published date': 'Mon, 05 Feb 2024 21:03:53 GMT', 'url': 'https://news.google.com/rss/articles/CBMiTGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzA1L3doeS10ZXNsYS1zdG9jay10YW5rZWQtaW4tamFudWFyeS_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': 'Tesla stock falls on Elon Musk drug use report, price target cut - Yahoo Finance', 'description': 'Tesla stock falls on Elon Musk drug use report, price target cut  Yahoo Finance', 'published date': 'Mon, 05 Feb 2024 21:10:54 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vdmlkZW8vdGVzbGEtc3RvY2stZmFsbHMtZWxvbi1tdXNrLTIxMTA1NDgwMy5odG1s0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'Analyst unveils new Tesla stock price target, focuses on crucial business - TheStreet', 'description': 'Analyst unveils new Tesla stock price target, focuses on crucial business  TheStreet', 'published date': 'Mon, 05 Feb 2024 21:48:31 GMT', 'url': 'https://news.google.com/rss/articles/CBMieWh0dHBzOi8vd3d3LnRoZXN0cmVldC5jb20vaW52ZXN0aW5nL3N0b2Nrcy9hbmFseXN0LXJldmVhbHMtbmV3LXRlc2xhLXByaWNlLXRhcmdldC13aXRoLWZvY3VzLW9uLXVuZGVydmFsdWVkLWJ1c2luZXNzLXVuaXTSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.thestreet.com', 'title': 'TheStreet'}}, {'title': 'Tesla stock is worst in S&P 500 to start 2024 - Quartz', 'description': 'Tesla stock is worst in S&P 500 to start 2024  Quartz', 'published date': 'Mon, 05 Feb 2024 22:26:40 GMT', 'url': 'https://news.google.com/rss/articles/CBMiOmh0dHBzOi8vcXouY29tL3Rlc2xhLXdvcnN0LXN0b2NrLXBlcmZvcm1lci1tdXNrLTE4NTEyMjc0MjbSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://qz.com', 'title': 'Quartz'}}, {'title': "Stock Market Today, Monday, Feb. 5, 2024: Dow Slides; Boeing, Meta, Nvidia, Catalent, Palantir, and More Market ... - Barron's", 'description': "Stock Market Today, Monday, Feb. 5, 2024: Dow Slides; Boeing, Meta, Nvidia, Catalent, Palantir, and More Market ...  Barron's", 'published date': 'Tue, 06 Feb 2024 00:25:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiPmh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2xpdmVjb3ZlcmFnZS9zdG9jay1tYXJrZXQtdG9kYXktMDIwNTI00gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': "Nvidia's Stock Surge Adds $500B of Value, Close to Tesla's Market Cap - Markets Insider", 'description': "Nvidia's Stock Surge Adds $500B of Value, Close to Tesla's Market Cap  Markets Insider", 'published date': 'Tue, 06 Feb 2024 12:19:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMic2h0dHBzOi8vbWFya2V0cy5idXNpbmVzc2luc2lkZXIuY29tL25ld3Mvc3RvY2tzL252aWRpYS10ZXNsYS1zdG9jay1wcmljZS1tYXJrZXQtY2FwLWh1YW5nLW11c2stYWktbWljcm9jaGlwcy0yMDI0LTLSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://markets.businessinsider.com', 'title': 'Markets Insider'}}, {'title': "Dow Jones Rises As Tesla Slides On Downgrade; Palantir Soars On Surging AI Demand - Investor's Business Daily", 'description': "Dow Jones Rises As Tesla Slides On Downgrade; Palantir Soars On Surging AI Demand  Investor's Business Daily", 'published date': 'Tue, 06 Feb 2024 14:49:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMidWh0dHBzOi8vd3d3LmludmVzdG9ycy5jb20vbWFya2V0LXRyZW5kL3N0b2NrLW1hcmtldC10b2RheS9kb3ctam9uZXMtZnV0dXJlcy1mYWxsLXBhbGFudGlyLXNvYXJzLW9uLXN1cmdpbmctYWktZGVtYW5kL9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investors.com', 'title': "Investor's Business Daily"}}, {'title': "Tesla Stock Is Down Again. Downgrades and Rates Aren’t Helping. - Barron's", 'description': "Tesla Stock Is Down Again. Downgrades and Rates Aren’t Helping.  Barron's", 'published date': 'Tue, 06 Feb 2024 15:14:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLXRvZGF5LTJhMTM1NjAx0gFFaHR0cHM6Ly93d3cuYmFycm9ucy5jb20vYW1wL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLXRvZGF5LTJhMTM1NjAx?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Tesla Now Worth $110 Billion Less Than Weight Loss Drug Maker Eli Lilly - Forbes', 'description': 'Tesla Now Worth $110 Billion Less Than Weight Loss Drug Maker Eli Lilly  Forbes', 'published date': 'Tue, 06 Feb 2024 15:30:39 GMT', 'url': 'https://news.google.com/rss/articles/CBMieWh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvZGVyZWtzYXVsLzIwMjQvMDIvMDYvdGVzbGEtbm93LXdvcnRoLTExMC1iaWxsaW9uLWxlc3MtdGhhbi13ZWlnaHQtbG9zcy1kcnVnLW1ha2VyLWVsaS1saWxseS_SAX1odHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2RlcmVrc2F1bC8yMDI0LzAyLzA2L3Rlc2xhLW5vdy13b3J0aC0xMTAtYmlsbGlvbi1sZXNzLXRoYW4td2VpZ2h0LWxvc3MtZHJ1Zy1tYWtlci1lbGktbGlsbHkvYW1wLw?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.forbes.com', 'title': 'Forbes'}}, {'title': 'Tesla’s stock is downgraded, more for Elon Musk’s issues than ‘tough’ EV demand - MarketWatch', 'description': 'Tesla’s stock is downgraded, more for Elon Musk’s issues than ‘tough’ EV demand  MarketWatch', 'published date': 'Tue, 06 Feb 2024 15:48:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMidWh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS90ZXNsYXMtc3RvY2staXMtZG93bmdyYWRlZC1tb3JlLWZvci1lbG9uLW11c2tzLWlzc3Vlcy10aGFuLXRvdWdoLWV2LWRlbWFuZC1kZjU1MDA1ZdIBeWh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9hbXAvc3RvcnkvdGVzbGFzLXN0b2NrLWlzLWRvd25ncmFkZWQtbW9yZS1mb3ItZWxvbi1tdXNrcy1pc3N1ZXMtdGhhbi10b3VnaC1ldi1kZW1hbmQtZGY1NTAwNWU?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.marketwatch.com', 'title': 'MarketWatch'}}, {'title': 'Where Will Tesla Stock Be in 2026? - The Motley Fool', 'description': 'Where Will Tesla Stock Be in 2026?  The Motley Fool', 'published date': 'Tue, 06 Feb 2024 16:15:57 GMT', 'url': 'https://news.google.com/rss/articles/CBMiTGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzA2L3doZXJlLXdpbGwtdGVzbGEtc3RvY2stYmUtaW4tMjAyNi_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': 'Why Eli Lilly Deserves to Unseat Tesla Among the Magnificent Seven - The Wall Street Journal', 'description': 'Why Eli Lilly Deserves to Unseat Tesla Among the Magnificent Seven  The Wall Street Journal', 'published date': 'Tue, 06 Feb 2024 17:22:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiamh0dHBzOi8vd3d3Lndzai5jb20vZmluYW5jZS9zdG9ja3Mvd2h5LWVsaS1saWxseS1kZXNlcnZlcy10by11bnNlYXQtdGVzbGEtYW1vbmctdGhlLW1hZ25pZmljZW50LTctODg3ZWRhN2LSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.wsj.com', 'title': 'The Wall Street Journal'}}, {'title': "The Story Elon Musk Doesn't Want to Talk About - Bloomberg", 'description': "The Story Elon Musk Doesn't Want to Talk About  Bloomberg", 'published date': 'Tue, 06 Feb 2024 21:54:43 GMT', 'url': 'https://news.google.com/rss/articles/CBMiaWh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vbmV3cy9hcnRpY2xlcy8yMDI0LTAyLTA2L3BvZGNhc3QtdGhlLXN0b3J5LWVsb24tbXVzay1kb2Vzbi10LXdhbnQtdG8tdGFsay1hYm91dNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.bloomberg.com', 'title': 'Bloomberg'}}, {'title': 'Tesla CEO Elon Musk needs to shut up, but not step down: Investor - Yahoo Finance', 'description': 'Tesla CEO Elon Musk needs to shut up, but not step down: Investor  Yahoo Finance', 'published date': 'Tue, 06 Feb 2024 23:04:30 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vdmlkZW8vdGVzbGEtY2VvLWVsb24tbXVzay1uZWVkcy0yMzA0MzA4NzQuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': "Where Will Tesla's Stock Be in 10 Years? - The Motley Fool", 'description': "Where Will Tesla's Stock Be in 10 Years?  The Motley Fool", 'published date': 'Wed, 07 Feb 2024 10:36:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiUWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzA3L3doZXJlLXdpbGwtdGVzbGFzLXN0b2NrLWJlLWluLTEwLXllYXJzL9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': "Three stocks that could replace Tesla in the 'Magnificent 7' - CNBC", 'description': "Three stocks that could replace Tesla in the 'Magnificent 7'  CNBC", 'published date': 'Wed, 07 Feb 2024 11:32:53 GMT', 'url': 'https://news.google.com/rss/articles/CBMiZmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDIvMDcvdGhyZWUtc3RvY2tzLXN0b2Nrcy10aGF0LWNvdWxkLXJlcGxhY2UtdGVzbGEtaW4tdGhlLW1hZ25pZmljZW50LTcuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.cnbc.com', 'title': 'CNBC'}}, {'title': 'TSLA Stock Forecast: Avoid Tesla Until It Reaches THIS Price - InvestorPlace', 'description': 'TSLA Stock Forecast: Avoid Tesla Until It Reaches THIS Price  InvestorPlace', 'published date': 'Wed, 07 Feb 2024 11:42:21 GMT', 'url': 'https://news.google.com/rss/articles/CBMiXmh0dHBzOi8vaW52ZXN0b3JwbGFjZS5jb20vMjAyNC8wMi90c2xhLXN0b2NrLWZvcmVjYXN0LWF2b2lkLXRlc2xhLXVudGlsLWl0LXJlYWNoZXMtdGhpcy1wcmljZS_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://investorplace.com', 'title': 'InvestorPlace'}}, {'title': 'Tesla: Buy, Sell, or Hold? - The Motley Fool', 'description': 'Tesla: Buy, Sell, or Hold?  The Motley Fool', 'published date': 'Wed, 07 Feb 2024 13:02:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzA3L3Rlc2xhLWJ1eS1zZWxsLW9yLWhvbGQv0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': 'Tesla (TSLA) Stokes Layoff Fears by Asking Whether Jobs Are Critical - Bloomberg', 'description': 'Tesla (TSLA) Stokes Layoff Fears by Asking Whether Jobs Are Critical  Bloomberg', 'published date': 'Wed, 07 Feb 2024 13:50:23 GMT', 'url': 'https://news.google.com/rss/articles/CBMicGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vbmV3cy9hcnRpY2xlcy8yMDI0LTAyLTA3L3Rlc2xhLXN0b2tlcy1sYXlvZmYtZmVhcnMtYnktYXNraW5nLXdoZXRoZXItam9icy1hcmUtY3JpdGljYWzSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.bloomberg.com', 'title': 'Bloomberg'}}, {'title': 'Deutsche Bank Recommends Buying Tesla and LI Stocks on the Dip - TipRanks.com - TipRanks', 'description': 'Deutsche Bank Recommends Buying Tesla and LI Stocks on the Dip - TipRanks.com  TipRanks', 'published date': 'Wed, 07 Feb 2024 17:56:19 GMT', 'url': 'https://news.google.com/rss/articles/CBMiW2h0dHBzOi8vd3d3LnRpcHJhbmtzLmNvbS9uZXdzL2RldXRzY2hlLWJhbmstcmVjb21tZW5kcy1idXlpbmctdGhlLWRpcC1vbi10c2xhLWFuZC1saS1zdG9ja3PSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.tipranks.com', 'title': 'TipRanks'}}, {'title': "What's Wrong With Tesla in 2024, and Will TSLA Stock Recover and Go Back Up? - Nasdaq", 'description': "What's Wrong With Tesla in 2024, and Will TSLA Stock Recover and Go Back Up?  Nasdaq", 'published date': 'Wed, 07 Feb 2024 18:30:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiaWh0dHBzOi8vd3d3Lm5hc2RhcS5jb20vYXJ0aWNsZXMvd2hhdHMtd3Jvbmctd2l0aC10ZXNsYS1pbi0yMDI0LWFuZC13aWxsLXRzbGEtc3RvY2stcmVjb3Zlci1hbmQtZ28tYmFjay11cNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.nasdaq.com', 'title': 'Nasdaq'}}, {'title': "Tesla Stock Rises. Layoffs May Be Coming. - Barron's", 'description': "Tesla Stock Rises. Layoffs May Be Coming.  Barron's", 'published date': 'Wed, 07 Feb 2024 20:12:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQ2h0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLWxheW9mZnMtYmEzNzg3YWLSAUdodHRwczovL3d3dy5iYXJyb25zLmNvbS9hbXAvYXJ0aWNsZXMvdGVzbGEtc3RvY2stcHJpY2UtbGF5b2Zmcy1iYTM3ODdhYg?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Cathie Wood is standing by Tesla amid sharp sell-off, betting on robotaxi cycle in 2 years - CNBC', 'description': 'Cathie Wood is standing by Tesla amid sharp sell-off, betting on robotaxi cycle in 2 years  CNBC', 'published date': 'Thu, 08 Feb 2024 00:50:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMifmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDIvMDcvY2F0aGllLXdvb2QtaXMtc3RhbmRpbmctYnktdGVzbGEtYW1pZC1zaGFycC1zZWxsLW9mZi1iZXR0aW5nLW9uLXJvYm90YXhpLWN5Y2xlLWluLTIteWVhcnMuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.cnbc.com', 'title': 'CNBC'}}, {'title': "How Much Profit is Hiding Within Tesla's Enormous $78 Billion in Automotive Revenue? - Yahoo Finance", 'description': "How Much Profit is Hiding Within Tesla's Enormous $78 Billion in Automotive Revenue?  Yahoo Finance", 'published date': 'Thu, 08 Feb 2024 11:22:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9tdWNoLXByb2ZpdC1oaWRpbmctd2l0aGluLXRlc2xhcy0xMTIyMDA0MjAuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': "Investors May Not View Tesla Stock As Part Of The Magnificent Seven Anymore. But Is TSLA A Buy? - Investor's Business Daily", 'description': "Investors May Not View Tesla Stock As Part Of The Magnificent Seven Anymore. But Is TSLA A Buy?  Investor's Business Daily", 'published date': 'Thu, 08 Feb 2024 13:12:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiZ2h0dHBzOi8vd3d3LmludmVzdG9ycy5jb20vbmV3cy9pcy10ZXNsYS1zdG9jay1hLWJ1eS1vci1hLXNlbGwtcmlnaHQtbm93LWFzLWVsb24tbXVzay10ZWFzZXMtY3liZXJ0cnVjay_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investors.com', 'title': "Investor's Business Daily"}}, {'title': "Rivian Stock Is Down More Than 30% But Will Unveil Its Next-Gen EV In March. Is RIVN A Buy Or A Sell? - Investor's Business Daily", 'description': "Rivian Stock Is Down More Than 30% But Will Unveil Its Next-Gen EV In March. Is RIVN A Buy Or A Sell?  Investor's Business Daily", 'published date': 'Thu, 08 Feb 2024 13:14:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMieWh0dHBzOi8vd3d3LmludmVzdG9ycy5jb20vbmV3cy9yaXZpYW4tc3RvY2stYS1idXktb3ItYS1zZWxsLXdpdGgtdGhpcy10ZXNsYS1idWxsLXNheWluZy1pdC1pcy1vbmUtb2YtdGhlLWNvcmUtZXYtcGxheWVycy_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investors.com', 'title': "Investor's Business Daily"}}, {'title': "There Is A Black Cloud Around Tesla's Stock - Jalopnik", 'description': "There Is A Black Cloud Around Tesla's Stock  Jalopnik", 'published date': 'Thu, 08 Feb 2024 15:35:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vamFsb3BuaWsuY29tL3RoZXJlLWlzLWEtYmxhY2stY2xvdWQtYXJvdW5kLXRlc2xhcy1zdG9jay0xODUxMjM4NjM40gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://jalopnik.com', 'title': 'Jalopnik'}}, {'title': "Tesla’s Chinese Production Falls. The Stock Rises. - Barron's", 'description': "Tesla’s Chinese Production Falls. The Stock Rises.  Barron's", 'published date': 'Thu, 08 Feb 2024 17:35:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiRmh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLWNoaW5hLWRlbGl2ZXJpZXMtNGE1MzIzOTTSAUpodHRwczovL3d3dy5iYXJyb25zLmNvbS9hbXAvYXJ0aWNsZXMvdGVzbGEtc3RvY2stY2hpbmEtZGVsaXZlcmllcy00YTUzMjM5NA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': '1 Artificial Intelligence (AI) Growth Stock to Buy Before It Soars 170%, According to a Wall Street Analyst - The Motley Fool', 'description': '1 Artificial Intelligence (AI) Growth Stock to Buy Before It Soars 170%, According to a Wall Street Analyst  The Motley Fool', 'published date': 'Fri, 09 Feb 2024 10:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzA5LzEtYWktc3RvY2stdG8tYnV5LWJlZm9yZS1pdC1zb2Fycy0xNzAtd2FsbC1zdHJlZXQv0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': 'Tesla, Inc. (TSLA) Is a Trending Stock: Facts to Know Before Betting on It - Yahoo Finance', 'description': 'Tesla, Inc. (TSLA) Is a Trending Stock: Facts to Know Before Betting on It  Yahoo Finance', 'published date': 'Fri, 09 Feb 2024 14:00:10 GMT', 'url': 'https://news.google.com/rss/articles/CBMiS2h0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy90ZXNsYS1pbmMtdHNsYS10cmVuZGluZy1zdG9jay0xNDAwMTAxMzQuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': "Here's how much Tesla stock is down since Cybertruck launch - Finbold - Finance in Bold", 'description': "Here's how much Tesla stock is down since Cybertruck launch  Finbold - Finance in Bold", 'published date': 'Fri, 09 Feb 2024 14:37:46 GMT', 'url': 'https://news.google.com/rss/articles/CBMiT2h0dHBzOi8vZmluYm9sZC5jb20vaGVyZXMtaG93LW11Y2gtdGVzbGEtc3RvY2staXMtZG93bi1zaW5jZS1jeWJlcnRydWNrLWxhdW5jaC_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finbold.com', 'title': 'Finbold - Finance in Bold'}}, {'title': 'Tesla stock sinks, Ozempic scares food CEOs: Business news roundup - Quartz', 'description': 'Tesla stock sinks, Ozempic scares food CEOs: Business news roundup  Quartz', 'published date': 'Fri, 09 Feb 2024 15:17:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiUGh0dHBzOi8vcXouY29tL3Rlc2xhLXN0b2NrLW96ZW1waWMtY2hhbmVsLXBhbGFudGlyLWhlYWx0aGNhcmUtbGF5b2Zmcy0xODUxMjQyNDU50gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://qz.com', 'title': 'Quartz'}}, {'title': "Magnificent Seven Stocks To Buy And Watch: Tesla Dives On Earnings - Investor's Business Daily", 'description': "Magnificent Seven Stocks To Buy And Watch: Tesla Dives On Earnings  Investor's Business Daily", 'published date': 'Fri, 09 Feb 2024 18:35:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiUWh0dHBzOi8vd3d3LmludmVzdG9ycy5jb20vcmVzZWFyY2gvbWFnbmlmaWNlbnQtc2V2ZW4tc3RvY2tzLXRvLWJ1eS1hbmQtYW5kLXdhdGNoL9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investors.com', 'title': "Investor's Business Daily"}}, {'title': "The tiny Chinese tea seller whose shares trade more than Tesla's - Financial Times", 'description': "The tiny Chinese tea seller whose shares trade more than Tesla's  Financial Times", 'published date': 'Fri, 09 Feb 2024 20:00:07 GMT', 'url': 'https://news.google.com/rss/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50L2MyYjY3NTgwLTdlMWItNDc0ZC05YzA2LWMzMDU1MTBiM2ZkNdIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.ft.com', 'title': 'Financial Times'}}, {'title': "Tesla Stock Rises for Fourth Day, a Winning Streak That Could Make It a Loser - Barron's", 'description': "Tesla Stock Rises for Fourth Day, a Winning Streak That Could Make It a Loser  Barron's", 'published date': 'Fri, 09 Feb 2024 21:10:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLXRvZGF5LWIyMWRjZDZh0gFFaHR0cHM6Ly93d3cuYmFycm9ucy5jb20vYW1wL2FydGljbGVzL3Rlc2xhLXN0b2NrLXByaWNlLXRvZGF5LWIyMWRjZDZh?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}, {'title': 'Tesla Inc. stock rises Friday, outperforms market - MarketWatch', 'description': 'Tesla Inc. stock rises Friday, outperforms market  MarketWatch', 'published date': 'Fri, 09 Feb 2024 21:31:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMia2h0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9kYXRhLW5ld3MvdGVzbGEtaW5jLXN0b2NrLXJpc2VzLWZyaWRheS1vdXRwZXJmb3Jtcy1tYXJrZXQtYjIxNzk4YzktNjg3N2UyMTk5NDVj0gFvaHR0cHM6Ly93d3cubWFya2V0d2F0Y2guY29tL2FtcC9kYXRhLW5ld3MvdGVzbGEtaW5jLXN0b2NrLXJpc2VzLWZyaWRheS1vdXRwZXJmb3Jtcy1tYXJrZXQtYjIxNzk4YzktNjg3N2UyMTk5NDVj?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.marketwatch.com', 'title': 'MarketWatch'}}, {'title': "Elon Musk's Tesla is in all kinds of trouble - Quartz", 'description': "Elon Musk's Tesla is in all kinds of trouble  Quartz", 'published date': 'Sat, 10 Feb 2024 09:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vcXouY29tL2VtYWlscy9xdWFydHotd2Vla2VuZC1icmllZi8xODUxMjM4MDY0L3Rlc2xhLXRyb3VibGVzLWVsb24tbXVza9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://qz.com', 'title': 'Quartz'}}, {'title': 'Tesla Was Once About Climate Change, Driverless Cars, AI. Now What? - The Wall Street Journal', 'description': 'Tesla Was Once About Climate Change, Driverless Cars, AI. Now What?  The Wall Street Journal', 'published date': 'Sat, 10 Feb 2024 10:30:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMicmh0dHBzOi8vd3d3Lndzai5jb20vYnVzaW5lc3MvYXV0b3MvdGVzbGEtaGFzLWxvbmctYmVlbi1hLWhvcGUtc3RvY2stYnV0LXdoYXQtYXJlLWludmVzdG9ycy1ob3BpbmctZm9yLW5vdy03MDMxMTcxZtIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.wsj.com', 'title': 'The Wall Street Journal'}}, {'title': "Don't Rely Solely on Auto Business for Tesla Stock, Advises Piper Sandler - TipRanks.com - TipRanks", 'description': "Don't Rely Solely on Auto Business for Tesla Stock, Advises Piper Sandler - TipRanks.com  TipRanks", 'published date': 'Sat, 10 Feb 2024 17:27:17 GMT', 'url': 'https://news.google.com/rss/articles/CBMiZWh0dHBzOi8vd3d3LnRpcHJhbmtzLmNvbS9uZXdzL2RvbnQtcmVseS1zb2xlbHktb24tYXV0by1idXNpbmVzcy1mb3ItdGVzbGEtc3RvY2stYWR2aXNlcy1waXBlci1zYW5kbGVy0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.tipranks.com', 'title': 'TipRanks'}}, {'title': 'AI year-end prediction for Tesla share price - Finbold - Finance in Bold', 'description': 'AI year-end prediction for Tesla share price  Finbold - Finance in Bold', 'published date': 'Sun, 11 Feb 2024 12:03:40 GMT', 'url': 'https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vZmluYm9sZC5jb20vYWkteWVhci1lbmQtcHJlZGljdGlvbi1mb3ItdGVzbGEtc2hhcmUtcHJpY2Uv0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finbold.com', 'title': 'Finbold - Finance in Bold'}}, {'title': 'Cathie Wood Just Keeps Buying More of This Magnificent AI Stock - Yahoo Finance', 'description': 'Cathie Wood Just Keeps Buying More of This Magnificent AI Stock  Yahoo Finance', 'published date': 'Sun, 11 Feb 2024 13:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiS2h0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9jYXRoaWUtd29vZC1qdXN0LWtlZXBzLWJ1eWluZy0xMzAwMDAyMzMuaHRtbNIBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': "Tesla's Slide Has Investors Wondering If It's Still Magnificent - Yahoo Finance", 'description': "Tesla's Slide Has Investors Wondering If It's Still Magnificent  Yahoo Finance", 'published date': 'Sun, 11 Feb 2024 13:47:55 GMT', 'url': 'https://news.google.com/rss/articles/CBMiVmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy90ZXNsYS1jb2xsYXBzZS1pbnZlc3RvcnMtd29uZGVyaW5nLXN0aWxsLTEzNDc1NTIxNi5odG1s0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': "Tesla (TSLA) Stock Value and Elon Musk's Magnificent Seven AI Rivals - Bloomberg", 'description': "Tesla (TSLA) Stock Value and Elon Musk's Magnificent Seven AI Rivals  Bloomberg", 'published date': 'Sun, 11 Feb 2024 14:00:18 GMT', 'url': 'https://news.google.com/rss/articles/CBMic2h0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vbmV3cy9hcnRpY2xlcy8yMDI0LTAyLTExL3Rlc2xhLWNvbGxhcHNlLWhhcy1pbnZlc3RvcnMtd29uZGVyaW5nLWlmLWl0LXMtc3RpbGwtbWFnbmlmaWNlbnTSAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.bloomberg.com', 'title': 'Bloomberg'}}, {'title': 'The Worst-Performing S&P 500 Stock in 2024: Will It Be Tesla? - InvestorPlace', 'description': 'The Worst-Performing S&P 500 Stock in 2024: Will It Be Tesla?  InvestorPlace', 'published date': 'Sun, 11 Feb 2024 14:45:33 GMT', 'url': 'https://news.google.com/rss/articles/CBMiXWh0dHBzOi8vaW52ZXN0b3JwbGFjZS5jb20vMjAyNC8wMi90aGUtd29yc3QtcGVyZm9ybWluZy1zcC01MDAtc3RvY2staW4tMjAyNC13aWxsLWl0LWJlLXRlc2xhL9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://investorplace.com', 'title': 'InvestorPlace'}}, {'title': 'The Truth About Tesla Stock: Should You Buy Now? - Yahoo Finance', 'description': 'The Truth About Tesla Stock: Should You Buy Now?  Yahoo Finance', 'published date': 'Sun, 11 Feb 2024 15:30:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiR2h0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy90cnV0aC10ZXNsYS1zdG9jay1idXktbm93LTE1MzAwMDM1Ny5odG1s0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finance.yahoo.com', 'title': 'Yahoo Finance'}}, {'title': 'The Truth About Tesla Stock: Should You Buy Now? - The Motley Fool', 'description': 'The Truth About Tesla Stock: Should You Buy Now?  The Motley Fool', 'published date': 'Sun, 11 Feb 2024 15:30:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiWWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzAyLzExL3RoZS10cnV0aC1hYm91dC10ZXNsYS1zdG9jay1zaG91bGQteW91LWJ1eS1ub3cv0gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.fool.com', 'title': 'The Motley Fool'}}, {'title': 'The Truth About Tesla Stock: Should You Buy Now? - The Globe and Mail', 'description': 'The Truth About Tesla Stock: Should You Buy Now?  The Globe and Mail', 'published date': 'Sun, 11 Feb 2024 15:49:41 GMT', 'url': 'https://news.google.com/rss/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWdsb2JlYW5kbWFpbC5jb20vaW52ZXN0aW5nL21hcmtldHMvc3RvY2tzL1RTTEEvcHJlc3NyZWxlYXNlcy8yMzk5Mzk5Ni_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.theglobeandmail.com', 'title': 'The Globe and Mail'}}, {'title': "Tesla Cuts Model Y Prices For A Limited Time — For Now - Investor's Business Daily", 'description': "Tesla Cuts Model Y Prices For A Limited Time — For Now  Investor's Business Daily", 'published date': 'Sun, 11 Feb 2024 16:00:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiXWh0dHBzOi8vd3d3LmludmVzdG9ycy5jb20vbmV3cy90ZXNsYS1jdXRzLW1vZGVsLXktcHJpY2VzLWZvci1hLWxpbWl0ZWQtdGltZS1hdC1sZWFzdC1mb3Itbm93L9IBAA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.investors.com', 'title': "Investor's Business Daily"}}, {'title': 'Does Tesla still belong in Magnificent Seven? - Fortune', 'description': 'Does Tesla still belong in Magnificent Seven?  Fortune', 'published date': 'Sun, 11 Feb 2024 17:19:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiUmh0dHBzOi8vZm9ydHVuZS5jb20vMjAyNC8wMi8xMS90ZXNsYS1tYWduaWZpY2VudC1zZXZlbi1haS1ldnMtaW52ZXN0aW5nLWVsb24tbXVzay_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://fortune.com', 'title': 'Fortune'}}, {'title': "Tesla Shareholders Also Are Frustrated With Delaware’s CEO Pay Decision - Barron's", 'description': "Tesla Shareholders Also Are Frustrated With Delaware’s CEO Pay Decision  Barron's", 'published date': 'Sun, 11 Feb 2024 20:17:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiYWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3Rlc2xhLXNoYXJlaG9sZGVycy1mcnVzdHJhdGVkLWRlbGF3YXJlLWNlby1wYXktZGVjaXNpb24tZDdmMTNlY2bSAWVodHRwczovL3d3dy5iYXJyb25zLmNvbS9hbXAvYXJ0aWNsZXMvdGVzbGEtc2hhcmVob2xkZXJzLWZydXN0cmF0ZWQtZGVsYXdhcmUtY2VvLXBheS1kZWNpc2lvbi1kN2YxM2VjZg?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://www.barrons.com', 'title': "Barron's"}}]


Rank the sentences from an article

In [ ]:

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
import heapq

def rank_sentences(text):
    stop_words = set(stopwords.words('english'))
    word_frequencies = defaultdict(int)
    
    # Tokenize words and count frequencies, ignoring stopwords
    for word in word_tokenize(text):
        if word.lower() not in stop_words:
            word_frequencies[word.lower()] += 1
    
    # Find the maximum frequency
    max_frequency = max(word_frequencies.values())
    
    # Normalize frequencies
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word] / max_frequency)
    
    sentence_scores = defaultdict(int)
    sentences = sent_tokenize(text)
    
    # Score sentences by summing normalized frequencies of their words
    for sent in sentences:
        for word in word_tokenize(sent.lower()):
            if word in word_frequencies:
                sentence_scores[sent] += word_frequencies[word]
    
    # Get top n sentences based on their scores
    summary_sentences = heapq.nlargest(3, sentence_scores, key=sentence_scores.get)
    return summary_sentences

from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_rank_sentences(text):
    sentences = sent_tokenize(text)
    tfidf_vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    sentence_scores = tfidf_matrix.sum(axis=1)

    # Convert matrix to flat list
    sentence_scores = [score[0,0] for score in sentence_scores]

    # Get top n sentences
    top_sentence_indices = sorted(((score, index) for index, score in enumerate(sentence_scores)), reverse=True)[:3]
    top_sentences = [sentences[index] for _, index in top_sentence_indices]

    return top_sentences

# Example usage
stock_news = load_articles_with_full_content("tesla_stock.jsonl")
print(len(stock_news))
key_sentences = tfidf_rank_sentences(stock_news[4]['content'])
for sentence in key_sentences:
    print(sentence)

News from other sources (Google)

In [1]:
# !pip3 install newspaper3k
import gnews
# from gnews import GNews
import importlib
importlib.reload(gnews)
from gnews import GNews
import json
from datetime import datetime
search_term = "Tesla stock"

google_news = GNews()
def get_news(search_terms):
    stock_news = google_news.get_news(search_terms)
    # Define a function to convert date strings into datetime objects
    def parse_date(date_str):
        # Example date: "Thu, 08 Feb 2024 15:35:00 GMT"
        return datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S GMT')

    # Sort stock_news by parsed 'publish date'
    stock_news = sorted(stock_news, key=lambda x: parse_date(x['published date']), reverse=True)
    
    print(stock_news)
    return stock_news

def get_content(url, homepage_url):
    article = google_news.get_full_article(url, homepage_url)
    return article

import json

def write_articles_to_file(articles, file_path):
    """Function to update or add articles while preserving the 'content' key for existing articles, without safeguarding the 'url'."""
    try:
        # Load existing articles from file
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_articles = {json.loads(line)['title'] + json.loads(line)['published date']: json.loads(line)
                                 for line in file}
    except FileNotFoundError:
        existing_articles = {}

    updated = False  # Flag to track if any article was updated or added

    for article in articles:
        key = article['title'] + article['published date']
        # Check if the article exists and preserve 'content' if it does
        if key in existing_articles and 'content' in existing_articles[key]:
            article['content'] = existing_articles[key]['content']
        # This updates or adds the new article, including updating 'url'
        existing_articles[key] = article
        updated = True

    # Write back to the file only if there was an update
    if updated:
        with open(file_path, 'w', encoding='utf-8') as file:
            for article in existing_articles.values():
                file.write(json.dumps(article, ensure_ascii=False) + '\n')

    return len(articles)  # Returning count of processed articles for simplicity

def load_articles_with_empty_content(file_path):
    """Load articles from a file, returning only those with empty or missing 'content'."""
    articles_with_empty_content = []

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                article = json.loads(line)
                # Check if 'content' is missing or empty
                if not article.get('content'):
                    articles_with_empty_content.append(article)
    except FileNotFoundError:
        print(f"No file found at {file_path}")
    
    return articles_with_empty_content

def load_articles_with_full_content(file_path):
    """Load articles from a file, returning only those with empty or missing 'content'."""
    articles_with_empty_content = []

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                article = json.loads(line)
                # Check if 'content' is missing or empty
                if article.get('content'):
                    articles_with_empty_content.append(article)
    except FileNotFoundError:
        print(f"No file found at {file_path}")
    
    return articles_with_empty_content

# stock_news = get_news(search_term)
# write_articles_to_file(stock_news, "tesla_stock.jsonl")
# stock_news = load_articles_with_empty_content("tesla_stock.jsonl")

# key_sentences = rank_sentences(article.text)
# for sentence in key_sentences:
#     print(sentence)


C:\Users\bow33\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
exclude = ['barrons.com', 'wsj.com']
stock_news = load_articles_with_empty_content("tesla_stock.jsonl")
print(len(stock_news))
for i, article in enumerate(stock_news):
    print(article['published date'])
    if any(excluded_domain in article['publisher']['href'] for excluded_domain in exclude):
        print("Closed due to  domain exclusion: ", article['publisher']['href'])
        continue
    article['content'] = get_content(article['url'], article['publisher']['href']).text
    article = [article]
    print("+_+_+_+_Content: ", article[0]['content'])
    
    write_articles_to_file(article, "tesla_stock.jsonl")
    print(i + 1, " out of ", len(stock_news))

51
Mon, 12 Feb 2024 16:07:00 GMT
Mon, 12 Feb 2024 15:36:14 GMT
Sun, 11 Feb 2024 20:17:00 GMT
Sat, 10 Feb 2024 10:30:00 GMT
Fri, 09 Feb 2024 21:10:00 GMT
Thu, 08 Feb 2024 17:35:00 GMT
Wed, 07 Feb 2024 20:12:00 GMT
Tue, 06 Feb 2024 15:14:00 GMT
Mon, 05 Feb 2024 17:09:00 GMT
Mon, 05 Feb 2024 16:52:00 GMT
Fri, 02 Feb 2024 05:02:46 GMT
Wed, 31 Jan 2024 08:00:00 GMT
Thu, 25 Jan 2024 08:00:00 GMT
Wed, 24 Jan 2024 08:00:00 GMT
Wed, 17 Jan 2024 08:00:00 GMT
Sat, 13 Jan 2024 08:00:00 GMT
Tue, 09 Jan 2024 08:00:00 GMT
Sat, 06 Jan 2024 08:00:00 GMT
Terminated (PID: 8400)
Terminated (PID: 27228)
Terminated (PID: 35432)
Terminated (PID: 41192)
Terminated (PID: 42900)
Terminated (PID: 43772)
+_+_+_+_Content:  We apologize, but your web browser is configured in such a way that it is preventing this site from implementing required components that protect your privacy and allow you to view and change your privacy settings. This functionality is required for privacy legislation in your region.

We recomm

WebDriverException: Message: no such execution context
  (Session info: chrome=121.0.6167.161)
Stacktrace:
	GetHandleVerifier [0x00721673+52979]
	(No symbol) [0x006A7961]
	(No symbol) [0x0058DC00]
	(No symbol) [0x0057F3BD]
	(No symbol) [0x0057D88F]
	(No symbol) [0x0057E0ED]
	(No symbol) [0x00589016]
	(No symbol) [0x00596F16]
	(No symbol) [0x0059A176]
	(No symbol) [0x0057E56D]
	(No symbol) [0x00596BCA]
	(No symbol) [0x005FB31D]
	(No symbol) [0x005E4286]
	(No symbol) [0x005BC063]
	(No symbol) [0x005BCECD]
	GetHandleVerifier [0x00A38D83+3294723]
	GetHandleVerifier [0x00A76CC2+3548482]
	GetHandleVerifier [0x00A71C9C+3527964]
	GetHandleVerifier [0x007B870E+671630]
	(No symbol) [0x006B1EB4]
	(No symbol) [0x006AD808]
	(No symbol) [0x006AD92D]
	(No symbol) [0x0069F7E0]
	BaseThreadInitThunk [0x762F7BA9+25]
	RtlInitializeExceptionChain [0x7716BD2B+107]
	RtlClearBits [0x7716BCAF+191]


In [ ]:
# import yfinance as yf
# import finnhub
# finnhub_client = finnhub.Client(api_key="cmr6gghr01qvmr5q06u0cmr6gghr01qvmr5q06ug")

# msft = yf.Ticker("MSFT")
# msft.news
# # Keeping only 'title' and 'link' in each item
# msft_news_filtered = [{"title": item["title"], "link": item["link"]} for item in msft.news]

# print(msft_news_filtered)

**FinForcaster App.py**


In [ ]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

# print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
import os
import re
import time
import json
import random
import finnhub
import torch
# import gradio as gr
import pandas as pd
import yfinance as yf
from pynvml import *
from peft import PeftModel
from collections import defaultdict
from datetime import date, datetime, timedelta
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

finnhub_client = finnhub.Client(api_key="cmr6gghr01qvmr5q06u0cmr6gghr01qvmr5q06ug")

def print_gpu_utilization():

    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def get_curday():

    return date.today().strftime("%Y-%m-%d")


def n_weeks_before(date_string, n):
    # date = datetime.strptime(date_string, "%Y-%m-%d") - timedelta(days=7*n)
    date = datetime.strptime(date_string, "%Y-%m-%d") - timedelta(days=n)
    return date.strftime("%Y-%m-%d")

def n_days_before(date_string, n):
    date = datetime.strptime(date_string, "%Y-%m-%d") - timedelta(days=n)
    return date.strftime("%Y-%m-%d")


def get_stock_data(stock_symbol, steps):

    stock_data = yf.download(stock_symbol, steps[0], steps[-1])
    if len(stock_data) == 0:
        raise gr.Error(f"Failed to download stock price data for symbol {stock_symbol} from yfinance!")

#     print(stock_data)

    dates, prices = [], []
    available_dates = stock_data.index.format()

    for date in steps[:-1]:
        for i in range(len(stock_data)):
            if available_dates[i] >= date:
                prices.append(stock_data['Close'][i])
                dates.append(datetime.strptime(available_dates[i], "%Y-%m-%d"))
                break

    dates.append(datetime.strptime(available_dates[-1], "%Y-%m-%d"))
    prices.append(stock_data['Close'][-1])

    return pd.DataFrame({
        "Start Date": dates[:-1], "End Date": dates[1:],
        "Start Price": prices[:-1], "End Price": prices[1:]
    })


def get_news(symbol, data, profile):
    
    news_list = []
    filtered_company_name = ' '.join(profile['name'].split()[:1])
    print(profile)

    for end_date, row in data.iterrows():
        start_date = row['Start Date'].strftime('%Y-%m-%d')
        end_date = row['End Date'].strftime('%Y-%m-%d')
#         print(symbol, ': ', start_date, ' - ', end_date)
        # time.sleep(1) # control qpm
        weekly_news = finnhub_client.company_news(symbol, _from=start_date, to=end_date)
        if len(weekly_news) == 0:
            raise gr.Error(f"No company news found for symbol {symbol} from finnhub!")
        weekly_news = [
            {
                "date": datetime.fromtimestamp(n['datetime']).strftime('%Y-%m-%d %H:%M'),
                "headline": n['headline'],
                "summary": n['summary'],
                "url": n['url'],
                "source": n['source']
            } 
            for n in weekly_news
            if "zacks.com" not in n['summary'].lower() and 
               (symbol.lower() in n['headline'].lower() or filtered_company_name.lower() in n['headline'].lower())
        ]
        weekly_news.sort(key=lambda x: x['date'])
        news_list.append(json.dumps(weekly_news))

    data['News'] = news_list

    # additions:
    total_news_count = sum(len(json.loads(weekly_news)) for weekly_news in news_list)
    print("Number of news items after filtering:", total_news_count) 

    # To save the news to a file, use:
    print_or_save_news(data['News'], f"{symbol}_news.txt")

    return data

def print_or_save_news(news_list, filename=None):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("")
    for news_items_str in news_list:
        news_items = json.loads(news_items_str)  # Load the JSON string into a list of dictionaries
        for news in news_items:
            news_str = f"Date: {news['date']}\nHeadline: {news['headline']}\nSummary: {news['summary']}\nURL: {news['url']}\nSource: {news['source']}\n\n"
            if filename:
                with open(filename, 'a', encoding='utf-8') as file:
                    file.write(news_str)
            else:
                print(news_str)

def get_company_prompt(symbol, profile):

    # profile = finnhub_client.company_profile2(symbol=symbol)
    if not profile:
        raise gr.Error(f"Failed to find company profile for symbol {symbol} from finnhub!")

    company_template = "[Company Introduction]:\n\n{name} is a leading entity in the {finnhubIndustry} sector. Incorporated and publicly traded since {ipo}, the company has established its reputation as one of the key players in the market. As of today, {name} has a market capitalization of {marketCapitalization:.2f} in {currency}, with {shareOutstanding:.2f} shares outstanding." \
        "\n\n{name} operates primarily in the {country}, trading under the ticker {ticker} on the {exchange}. As a dominant force in the {finnhubIndustry} space, the company continues to innovate and drive progress within the industry."

    formatted_str = company_template.format(**profile)

    return formatted_str


def get_prompt_by_row(symbol, row):

    start_date = row['Start Date'] if isinstance(row['Start Date'], str) else row['Start Date'].strftime('%Y-%m-%d')
    end_date = row['End Date'] if isinstance(row['End Date'], str) else row['End Date'].strftime('%Y-%m-%d')
    term = 'increased' if row['End Price'] > row['Start Price'] else 'decreased'
    head = "From {} to {}, {}'s stock price {} from {:.2f} to {:.2f}. Company news during this period are listed below:\n\n".format(
        start_date, end_date, symbol, term, row['Start Price'], row['End Price'])

    news = json.loads(row["News"])
    news = ["[Headline]: {}\n[Summary]: {}\n".format(
        n['headline'], n['summary']) for n in news if n['date'][:8] <= end_date.replace('-', '') and \
        not n['summary'].startswith("Looking for stock market analysis and research with proves results?")]

    basics = json.loads(row['Basics'])
    if basics:
        basics = "Some recent basic financials of {}, reported at {}, are presented below:\n\n[Basic Financials]:\n\n".format(
            symbol, basics['period']) + "\n".join(f"{k}: {v}" for k, v in basics.items() if k != 'period')
    else:
        basics = "[Basic Financials]:\n\nNo basic financial reported."

    return head, news, basics


def sample_news(news, k=5):

    return [news[i] for i in sorted(random.sample(range(len(news)), k))]


def get_current_basics(symbol, curday):

    basic_financials = finnhub_client.company_basic_financials(symbol, 'all')
    if not basic_financials['series']:
        raise gr.Error(f"Failed to find basic financials for symbol {symbol} from finnhub!")

    final_basics, basic_list, basic_dict = [], [], defaultdict(dict)

    for metric, value_list in basic_financials['series']['quarterly'].items():
        for value in value_list:
            basic_dict[value['period']].update({metric: value['v']})

    for k, v in basic_dict.items():
        v.update({'period': k})
        basic_list.append(v)

    basic_list.sort(key=lambda x: x['period'])

    for basic in basic_list[::-1]:
        if basic['period'] <= curday:
            break

    return basic


def get_all_prompts_online(symbol, data, curday, profile, with_basics=True):

    company_prompt = get_company_prompt(symbol, profile)

    prev_rows = []

    for row_idx, row in data.iterrows():
        head, news, _ = get_prompt_by_row(symbol, row)
        prev_rows.append((head, news, None))

    prompt = ""
    for i in range(-len(prev_rows), 0):
        prompt += "\n" + prev_rows[i][0]
        sampled_news = sample_news(
            prev_rows[i][1],
            min(5, len(prev_rows[i][1]))
        )
        if sampled_news:
            prompt += "\n".join(sampled_news)
        else:
            prompt += "No relative news reported."

    period = "{} to {}".format(curday, n_weeks_before(curday, -1))

    if with_basics:
        basics = get_current_basics(symbol, curday)
        basics = "Some recent basic financials of {}, reported at {}, are presented below:\n\n[Basic Financials]:\n\n".format(
            symbol, basics['period']) + "\n".join(f"{k}: {v}" for k, v in basics.items() if k != 'period')
    else:
        basics = "[Basic Financials]:\n\nNo basic financial reported."

    info = company_prompt + '\n' + prompt + '\n' + basics
    prompt = info + f"\n\nBased on all the information before {curday}, let's first analyze the positive developments and potential concerns for {symbol}. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. " \
        f"Then make your prediction of the {symbol} stock price movement for next week ({period}). Provide a summary analysis to support your prediction."

    return info, prompt


def construct_prompt(ticker, curday, n_weeks, use_basics):

    profile = finnhub_client.company_profile2(symbol=ticker)

    try:
        steps = [n_weeks_before(curday, n) for n in range(n_weeks + 1)][::-1]
    except Exception:
        raise gr.Error(f"Invalid date {curday}!")

    data = get_stock_data(ticker, steps)
    data = get_news(ticker, data, profile)
    data['Basics'] = [json.dumps({})] * len(data)
    # print(data)

    info, prompt = get_all_prompts_online(ticker, data, curday, profile, use_basics)

    prompt = B_INST + B_SYS + SYSTEM_PROMPT + E_SYS + prompt + E_INST
    # print(prompt)

    return info, prompt


def predict(ticker, date, n_weeks, use_basics):

    print_gpu_utilization()

    info, prompt = construct_prompt(ticker, date, n_weeks, use_basics)

    # additions:
    # save prompt to a file
    with open("./prompt.txt", 'w', encoding='utf-8') as file:
        file.write(prompt)

    inputs = tokenizer(
        prompt, return_tensors='pt', padding=False
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    print("Inputs loaded onto devices.")

    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True, streamer=streamer
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)

    torch.cuda.empty_cache()

    return info, answer, output

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload/"
)
model = PeftModel.from_pretrained(
    base_model,
    'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora',
    offload_folder="offload/"
)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    )

streamer = TextStreamer(tokenizer)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

SYSTEM_PROMPT = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. " \
    "Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ..."

In [ ]:
count = 0
def extract_and_append_prediction_to_file(date, offset, option, ticker):
    # Define the pattern to search for
    pattern = r"(Down by \d+-\d+%|Up by \d+-\d+%)"
    # Find all occurrences of the pattern
    matches = re.findall(pattern, answer)
    # Open the file in append mode and write the matches
    with open(f"{ticker}_from:{date}_past_{option}:{offset}.txt", 'a', encoding='utf-8') as file:
        for match in matches:
            file.write(match + '\n')
            print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ prediction")

def extract_and_append_reasoning_to_file(output, date, offset, option, ticker):
    # Open the file in append mode and write the matches
    with open(f"{ticker}_reasoning_from:{date}_past_{option}:{offset}.txt", 'a', encoding='utf-8') as file:
        file.write(output + '\n')
        file.write('-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+\n')
        print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ output")

# Run the process 10 times
for _ in range(10):
    ticker = "TSLA"
    date = "2024-01-24" # get_curday()
    offset = 1

    option = {
        "days": "days",
        "weeks": "weeks"
    }
    option = "days"

    text, answer, output = predict(ticker, date, offset, False)
    extract_and_append_prediction_to_file(date, offset, option, ticker)
    extract_and_append_reasoning_to_file(output, date, offset, option, ticker)
    count+=1
    print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+", count)

In [ ]:
def read_and_print_file(filename):
    try:
        with open(filename, 'r') as file:
            print(f"Contents of {filename}:")
            for line in file:
                print(line.strip())  # .strip() removes any leading/trailing whitespace, including newline characters
    except FileNotFoundError:
        print(f"File {filename} not found.")

# Example usage
filename = "TSLA_from:2024-01-24_past_days:1.txt"
#           TSLA_from:2023-11-9_past_days:3.txt
read_and_print_file(filename)

Find key sentences